In [25]:
import os
import polars as pl
import re
pl.Config.set_tbl_rows(100)  

csv_path = "../../data/csv_raw"
cleaned_csv_path ="../../data/csv_cleaned"

In [26]:
def show_nb_columns_per_file(csv_folder_path: str):
    counts = {}
    csv_filenames = os.listdir(csv_folder_path)
    
    for filename in csv_filenames:
        file_path = os.path.join(csv_folder_path, filename)

        df = pl.read_csv(file_path, infer_schema=False)
        counts[filename] = len(df.columns)


    # Sort files by number of columns
    sorted_counts = sorted(counts.items(), key=lambda x: x[1])

    # Print results
    for filename, column_count in sorted_counts:
        print(f"{filename}: {column_count}")

# Call the function with the CSV folder path
show_nb_columns_per_file(csv_path)

1813_part1.csv: 35
1805.csv: 39
1808.csv: 39
1810.csv: 39
1806.csv: 39
1807.csv: 39
1809.csv: 39
1813_part2.csv: 47
1895.csv: 52
1849.csv: 52
1853.csv: 52
1850.csv: 52
1854.csv: 52
1856.csv: 52
1851.csv: 52
1852.csv: 52
1857.csv: 52
1858.csv: 52
1843.csv: 53
1884.csv: 53
1861.csv: 53
1865.csv: 53
1877.csv: 53
1870.csv: 53
1885.csv: 53
1874.csv: 53
1872.csv: 53
1832.csv: 53
1873.csv: 53
1866.csv: 53
1844.csv: 53
1846.csv: 53
1848.csv: 53
1845.csv: 53
1882.csv: 53
1835.csv: 53
1837.csv: 53
1860.csv: 53
1871.csv: 53
1847.csv: 53
1839.csv: 53
1841.csv: 53
1838.csv: 53
1878.csv: 53
1875.csv: 53
1869.csv: 53
1879.csv: 53
1876.csv: 53
1883.csv: 53
1867.csv: 53
1862.csv: 53
1836.csv: 53
1863.csv: 53
1840.csv: 53
1868.csv: 53
1842.csv: 53
1864.csv: 53
1880.csv: 53
1881.csv: 53
1859.csv: 53
1894.csv: 58
1896.csv: 58
1893.csv: 58
1897.csv: 58
1888.csv: 58
1889.csv: 58
1890.csv: 58
1891.csv: 58
1892.csv: 58
1887.csv: 58
1886.csv: 58
1898.csv: 58
1855.csv: 82


In [27]:
def get_perfect_match_columns(csv_folder_path: str):
    counts = {}

    csv_filenames = os.listdir(csv_folder_path)
    for filename in csv_filenames:
        # Polar can infer types but as the columns may not be clean we force polar to read every column as string
        df = pl.read_csv(f"{csv_folder_path}/{filename}", infer_schema=False)

        for col in df.columns:
            counts.setdefault(col, [])
            counts[col].append(filename)
    
    length = len(csv_filenames)
    return [col for col, files in counts.items() if len(files) == length]
            
def show_columns(csv_folder_path: str):
    counts = {}

    csv_filenames = os.listdir(csv_folder_path)
    for filename in csv_filenames:
        # Polar can infer types but as the columns may not be clean we force polar to read every column as string
        df = pl.read_csv(f"{csv_folder_path}/{filename}", infer_schema=False)

        for col in df.columns:
            counts.setdefault(col, [])
            counts[col].append(filename)
    
    length = len(csv_filenames)
    print("Perfect match: ")
    for col, files in counts.items():
        if len(files) == length:
            print(f" - {col}")

    print("\nOnly partial match")
    for col, files in counts.items():
        if len(files) != length:
            print(f" - {col}: {len(files)}")


show_columns(csv_path)


Perfect match: 
 - nom_rue
 - nom_rue_htr_corr
 - nom_rue_norm
 - no_maison
 - chef_prenom
 - chef_prenom_htr_corr
 - chef_prenom_norm
 - chef_nom
 - chef_nom_htr_corr
 - chef_nom_norm
 - chef_origine
 - chef_origine_htr_corr
 - chef_origine_norm
 - chef_vocation
 - chef_vocation_htr_corr
 - chef_vocation_norm
 - observations
 - Page

Only partial match
 - proprietaire_nom: 60
 - proprietaire_nom_corr: 57
 - proprietaire_nom_htr_corr: 60
 - proprietaire_nom_norm: 60
 - chef_nom_corr: 69
 - chef_annee_naissance: 60
 - epouse_nom: 70
 - epouse_nom_corr: 66
 - epouse_nom_htr_corr: 70
 - epouse_nom_norm: 70
 - epouse_annee_naissance: 60
 - enfants_dans_la_commune_prenom: 30
 - enfants_dans_la_commune_prenom_htr_corr: 30
 - enfants_dans_la_commune_prenom_norm: 30
 - enfants_annee_naissance: 53
 - chef_origine_corr: 69
 - chef_annee_arrivee: 23
 - chef_vocation_top_terms: 72
 - chef_recepisse: 53
 - pensionnaires_prenom: 66
 - pensionnaires_prenom_htr_corr: 66
 - pensionnaires_prenom_norm: 6

In [28]:
def filter_files_by_column(csv_folder_path: str, col_name: str, must_have: bool = True):
    """
    Filters files in a folder based on whether they contain a specific column.
    
    :param csv_folder_path: Path to the folder containing CSV files.
    :param col_name: Column name to check for.
    :param must_have: If True, return files that contain the column. If False, return files that do not contain the column.
    :return: List of filenames matching the condition.
    """
    csv_filenames = os.listdir(csv_folder_path)
    matching_files = []
    
    for filename in csv_filenames:
        file_path = os.path.join(csv_folder_path, filename)
        try:
            df = pl.read_csv(file_path, infer_schema=False)
            if (col_name in df.columns) == must_have:
                matching_files.append(filename)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
    
    return matching_files

def show_files_with_column(csv_folder_path: str, col_name: str):
    print(filter_files_by_column(csv_folder_path, col_name, must_have=True))

def show_files_without_column(csv_folder_path: str, col_name: str):
    print(filter_files_by_column(csv_folder_path, col_name, must_have=False))

def show_years_in_common(csv_folder_path: str, col_names: list[str]):
    if not col_names:
        return print([])

    common_files = set(filter_files_by_column(csv_folder_path, col_names[0], must_have=True))
    for col_name in col_names[1:]:
        common_files.intersection_update(filter_files_by_column(csv_folder_path, col_name, must_have=True))
    
    print(list(common_files))
    
# 1883, 1884, 1885 
show_files_without_column(csv_path,"epouse_nom" )
# 1883, 1884, 1885 
show_files_with_column(csv_path, "epouse_nom_norm")






['1884.csv', '1885.csv', '1883.csv']
['1843.csv', '1895.csv', '1861.csv', '1865.csv', '1877.csv', '1894.csv', '1870.csv', '1849.csv', '1896.csv', '1874.csv', '1872.csv', '1832.csv', '1873.csv', '1805.csv', '1866.csv', '1844.csv', '1846.csv', '1848.csv', '1845.csv', '1893.csv', '1897.csv', '1853.csv', '1850.csv', '1808.csv', '1854.csv', '1888.csv', '1813_part1.csv', '1855.csv', '1882.csv', '1835.csv', '1837.csv', '1860.csv', '1810.csv', '1871.csv', '1847.csv', '1839.csv', '1841.csv', '1838.csv', '1878.csv', '1875.csv', '1856.csv', '1869.csv', '1889.csv', '1879.csv', '1876.csv', '1851.csv', '1867.csv', '1806.csv', '1807.csv', '1852.csv', '1862.csv', '1836.csv', '1890.csv', '1857.csv', '1863.csv', '1840.csv', '1868.csv', '1842.csv', '1864.csv', '1891.csv', '1892.csv', '1880.csv', '1887.csv', '1809.csv', '1881.csv', '1858.csv', '1813_part2.csv', '1886.csv', '1859.csv', '1898.csv']


In [29]:
# Before manually cleaning no year has both columns => expect to be the same column but with different names
show_years_in_common(csv_path, ["enfants_dans_la_commune_prenom","enfants_chez_parents_prenom"] )
show_years_in_common(csv_path, ["enfants_dans_la_commune_prenom", "fils_prenom"])
show_years_in_common(csv_path, ["enfants_chez_parents_prenom", "fils_prenom"])

show_files_with_column(csv_path, "enfants_dans_la_commune_prenom")
show_files_with_column(csv_path, "enfants_chez_parents_prenom")
show_files_with_column(csv_path, "fils_prenom")
show_files_with_column(csv_path, "filles_prenom")


[]
[]
[]
['1843.csv', '1861.csv', '1849.csv', '1832.csv', '1844.csv', '1846.csv', '1848.csv', '1845.csv', '1853.csv', '1850.csv', '1854.csv', '1855.csv', '1835.csv', '1837.csv', '1860.csv', '1847.csv', '1839.csv', '1841.csv', '1838.csv', '1856.csv', '1851.csv', '1852.csv', '1862.csv', '1836.csv', '1857.csv', '1840.csv', '1842.csv', '1858.csv', '1813_part2.csv', '1859.csv']
['1884.csv', '1895.csv', '1865.csv', '1877.csv', '1894.csv', '1870.csv', '1885.csv', '1896.csv', '1874.csv', '1872.csv', '1873.csv', '1866.csv', '1893.csv', '1897.csv', '1888.csv', '1882.csv', '1871.csv', '1878.csv', '1875.csv', '1869.csv', '1889.csv', '1879.csv', '1876.csv', '1883.csv', '1867.csv', '1890.csv', '1863.csv', '1868.csv', '1864.csv', '1891.csv', '1892.csv', '1880.csv', '1887.csv', '1881.csv', '1886.csv', '1898.csv']
['1805.csv', '1808.csv', '1813_part1.csv', '1810.csv', '1806.csv', '1807.csv', '1809.csv']
['1805.csv', '1808.csv', '1813_part1.csv', '1810.csv', '1806.csv', '1807.csv', '1809.csv']


## Cleaning

In [30]:
def columns_span_all_years(csv_folder_path: str, columns: list[str]):
    if len(columns) == 0:
        return False
    
    a = set(filter_files_by_column(csv_folder_path, columns[0], True))
    for col in columns[1:]:
        a.update(filter_files_by_column(csv_folder_path, col, True))
        
    csv_filenames = os.listdir(csv_folder_path)
    missing_files = []

    for filename in csv_filenames:
        if filename not in a:
            missing_files.append(filename)
    
    return missing_files

cols = ["enfants_dans_la_commune_prenom","enfants_chez_parents_prenom","fils_prenom"]
print(f"Columns span all years: {len(columns_span_all_years(csv_path, cols)) == 0}")

Columns span all years: True


In [31]:
def rename_columns(csv_folder_path: str, new_folder_path: str, renaming: dict):
    csv_filenames = os.listdir(csv_folder_path)
    for filename in csv_filenames:
        file_path = os.path.join(csv_folder_path, filename)
        try:
            df = pl.read_csv(file_path, infer_schema=False)
            for (old_name, new_name) in renaming.items():
                if old_name in df.columns:
                    df = df.rename({old_name: new_name})
                    
            new_file_path = os.path.join(new_folder_path, filename)
            df.write_csv(new_file_path)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            

# Rename columns
def normalize_child_columns(csv_folder_path: str, new_folder_path: str):
    csv_filenames = os.listdir(csv_folder_path)
    for filename in csv_filenames:
        file_path = os.path.join(csv_folder_path, filename)
        try:
            df = pl.read_csv(file_path, infer_schema=False)
            
            if "fils_prenom" in df.columns and "filles_prenom" in df.columns:
                fils = df["fils_prenom"].fill_null("")
                filles = df["filles_prenom"].fill_null("")
                df = df.with_columns(
                    (fils + "|" + filles).str.strip_chars("|").alias("enfants_chez_parents_prenom")
                )
                df = df.with_columns(
                    (fils + "|" + filles).str.strip_chars("|").alias("enfants_chez_parents_prenom_htr_corr")
                )
                df = df.with_columns(
                    (fils + "|" + filles).str.strip_chars("|").alias("enfants_chez_parents_prenom_norm")
                )
                df = df.drop([col for col in ["fils_prenom", "filles_prenom"] if col in df.columns])
            
            new_file_path = os.path.join(new_folder_path, filename)
            df.write_csv(new_file_path)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

renaming = {
    "enfants_dans_la_commune_prenom":"enfants_chez_parents_prenom",
    "enfants_dans_la_commune_prenom_htr_corr":"enfants_chez_parents_prenom_htr_corr",
    "enfants_dans_la_commune_prenom_norm": "enfants_chez_parents_prenom_norm"
}





In [32]:
# To be executed once
#normalize_child_columns(csv_path, cleaned_csv_path)
#rename_columns(cleaned_csv_path, cleaned_csv_path, renaming)

In years 1883, 1884 and 1885 rename the columns:
* `epouse_nom_prenom_norm` to `epouse_nom_norm`
* `epouse_nom_prenom` to `epouse_nom`
* `epouse_nom_prenom_corr` to `epouse_nom_corr`
* `epouse_nom_prenom_htr_corr`to `epouse_nom_htr_corr`

In [33]:
renaming = {
    "epouse_nom_prenom_norm":"epouse_nom_norm",
    "epouse_nom_prenom":"epouse_nom",
    "epouse_nom_prenom_corr": "epouse_nom_corr",
    "epouse_nom_prenom_htr_corr":"epouse_nom_htr_corr"
}
# To be executed once
#rename_columns(cleaned_csv_path, cleaned_csv_path, renaming)


In [34]:
show_columns(cleaned_csv_path)
show_files_without_column(cleaned_csv_path, "epouse_nom_corr")

Perfect match: 
 - nom_rue
 - nom_rue_htr_corr
 - nom_rue_norm
 - no_maison
 - chef_prenom
 - chef_prenom_htr_corr
 - chef_prenom_norm
 - chef_nom
 - chef_nom_htr_corr
 - chef_nom_norm
 - epouse_nom
 - epouse_nom_htr_corr
 - epouse_nom_norm
 - enfants_chez_parents_prenom
 - enfants_chez_parents_prenom_htr_corr
 - enfants_chez_parents_prenom_norm
 - chef_origine
 - chef_origine_htr_corr
 - chef_origine_norm
 - chef_vocation
 - chef_vocation_htr_corr
 - chef_vocation_norm
 - observations
 - Page

Only partial match
 - proprietaire_nom: 62
 - proprietaire_nom_corr: 58
 - proprietaire_nom_htr_corr: 62
 - proprietaire_nom_norm: 62
 - chef_nom_corr: 70
 - chef_annee_naissance: 62
 - epouse_nom_corr: 70
 - epouse_annee_naissance: 62
 - enfants_annee_naissance: 52
 - chef_origine_corr: 70
 - chef_annee_arrivee: 25
 - chef_vocation_top_terms: 74
 - chef_recepisse: 52
 - pensionnaires_prenom: 65
 - pensionnaires_prenom_htr_corr: 65
 - pensionnaires_prenom_norm: 65
 - pensionnaires_nom: 65
 - pen

In [35]:
# Pensionnaires
cols = ["pensionnaires_nom","nb_pensionnaires"]
print(f"Columns span all years: {len(columns_span_all_years(csv_path, cols)) == 0}")
show_files_without_column(csv_path, "pensionnaires_nom")
show_files_with_column(csv_path, "nb_pensionnaires")

Columns span all years: True
['1805.csv', '1808.csv', '1813_part1.csv', '1810.csv', '1806.csv', '1807.csv', '1809.csv']
['1805.csv', '1808.csv', '1813_part1.csv', '1810.csv', '1806.csv', '1807.csv', '1809.csv']


Pensionnaires_nom and nb_pensionnaires span all years but they cannot be unified as nb_pensionnaires contains the number of pensionnaires and pensionnaires_nom contain their names. (1813 is a special case see below)

The pensionaire_nom column can be used after the year 1813

Merge manually 1813_part1 and 1813_part2 together

In [36]:
def show_columns_in_common(csv_path1: str, csv_path2: str):
    df1 = pl.read_csv(csv_path1, infer_schema=False)
    df2 = pl.read_csv(csv_path2, infer_schema=False)

    print([col for col in df1.columns if col in df2.columns])
    
        

def show_columns_not_in_common(csv_path1: str, csv_path2: str):
    df1 = pl.read_csv(csv_path1, infer_schema=False)
    df2 = pl.read_csv(csv_path2, infer_schema=False)

    print([col for col in df1.columns if col not in df2.columns])
    print([col for col in df2.columns if col not in df1.columns])

show_columns_in_common("../../data/csv_cleaned/1813_part1.csv", "../../data/csv_cleaned/1813_part2.csv")   
show_columns_not_in_common("../../data/csv_cleaned/1813_part1.csv", "../../data/csv_cleaned/1813_part2.csv")   




['nom_rue', 'nom_rue_htr_corr', 'nom_rue_norm', 'no_maison', 'proprietaire_nom', 'proprietaire_nom_htr_corr', 'proprietaire_nom_norm', 'chef_prenom', 'chef_prenom_htr_corr', 'chef_prenom_norm', 'chef_nom', 'chef_nom_htr_corr', 'chef_nom_norm', 'chef_annee_naissance', 'epouse_nom', 'epouse_nom_htr_corr', 'epouse_nom_norm', 'epouse_annee_naissance', 'enfants_chez_parents_prenom', 'enfants_chez_parents_prenom_htr_corr', 'enfants_chez_parents_prenom_norm', 'chef_origine', 'chef_origine_htr_corr', 'chef_origine_norm', 'chef_annee_arrivee', 'chef_vocation', 'chef_vocation_top_terms', 'chef_vocation_htr_corr', 'chef_vocation_norm', 'observations', 'Page']
[]
[]


In this case the nb_pensionnaires column contain the first and second name of the pensionaire so the pensionnaires_prenom and pensionnaires_nom could be recreated, but for simplicity the columns were removed.

Manually delete from 1813_part1.csv:
* fils_annee_naissance
* filles_annee_naissance
* nb_domestiques
* nb_ouvriers
* nb_pensionnaires 

Maunally added in 1813_part1.csv 
* enfants_dans_la_commune_prenom_htr_corr
* enfants_dans_la_commune_prenom_norm

Manually deleted:
* enfants_annee_naissance 
* chef_recepisse 
* pensionnaires_prenom
* pensionnaires_prenom_htr_corr
* pensionnaires_prenom_norm
* pensionnaires_nom
* pensionnaires_nom_htr_corr
* pensionnaires_nom_norm
* pensionnaires_origine
* pensionnaires_origine_htr_corr
* pensionnaires_origine_norm
* pensionnaires_condition
* pensionnaires_condition_top_terms
* pensionnaires_condition_htr_corr
* pensionnaires_condition_norm
* pensionnaires_recepisse'

## In memory data cleaning

In [37]:
show_columns(cleaned_csv_path)

Perfect match: 
 - nom_rue
 - nom_rue_htr_corr
 - nom_rue_norm
 - no_maison
 - chef_prenom
 - chef_prenom_htr_corr
 - chef_prenom_norm
 - chef_nom
 - chef_nom_htr_corr
 - chef_nom_norm
 - epouse_nom
 - epouse_nom_htr_corr
 - epouse_nom_norm
 - enfants_chez_parents_prenom
 - enfants_chez_parents_prenom_htr_corr
 - enfants_chez_parents_prenom_norm
 - chef_origine
 - chef_origine_htr_corr
 - chef_origine_norm
 - chef_vocation
 - chef_vocation_htr_corr
 - chef_vocation_norm
 - observations
 - Page

Only partial match
 - proprietaire_nom: 62
 - proprietaire_nom_corr: 58
 - proprietaire_nom_htr_corr: 62
 - proprietaire_nom_norm: 62
 - chef_nom_corr: 70
 - chef_annee_naissance: 62
 - epouse_nom_corr: 70
 - epouse_annee_naissance: 62
 - enfants_annee_naissance: 52
 - chef_origine_corr: 70
 - chef_annee_arrivee: 25
 - chef_vocation_top_terms: 74
 - chef_recepisse: 52
 - pensionnaires_prenom: 65
 - pensionnaires_prenom_htr_corr: 65
 - pensionnaires_prenom_norm: 65
 - pensionnaires_nom: 65
 - pen

In [ ]:
pl.Config.set_tbl_rows(100)  

def replace_entries_with_placeholder(df: pl.DataFrame, column_name: str, pattern: str) -> pl.DataFrame:
    return df.with_columns(df[column_name].cast(pl.Utf8).str.strip_chars().str.replace_all(pattern, "").replace("",None))

def clean_non_numeric_entries(df: pl.DataFrame, column_name: str) -> pl.DataFrame:
    return replace_entries_with_placeholder(df, column_name, r"[^0-9.]")

def clean_dots(df:pl.DataFrame, column_name:str) -> pl.DataFrame:
    return replace_entries_with_placeholder(df, column_name, r"[·]")

def separated_with(
        df: pl.DataFrame, column_name: str, separators: list[str]
    ) -> pl.DataFrame:

        # Create a regex pattern to match any of the given separators
        pattern = "|".join(map(re.escape, separators))

        df = df.with_columns(
            df[column_name]
            .cast(pl.Utf8)  
            .str.replace_all(rf"\s*({pattern})\s*", "|") 
            .str.split("|")  # Split into lists
            .list.eval(pl.element().str.strip_chars()) 
        )
        return df


df = pl.DataFrame({"names": [" Alice |  Bob  , Charlie "," Francis ", " F | de La Roche"], "dates":["          1823 ", "·", "1900"],"street":[" here o        ", "·","there "]})
df = separated_with(df, "names", ["|",","])
df = clean_non_numeric_entries(df, "dates")
df = clean_dots(df, "street")
print(df)


shape: (3, 3)
┌─────────────────────────────┬───────┬────────┐
│ names                       ┆ dates ┆ street │
│ ---                         ┆ ---   ┆ ---    │
│ list[str]                   ┆ str   ┆ str    │
╞═════════════════════════════╪═══════╪════════╡
│ ["Alice", "Bob", "Charlie"] ┆ 1823  ┆ here o │
│ ["Francis"]                 ┆ null  ┆ null   │
│ ["F", "de La Roche"]        ┆ 1900  ┆ there  │
└─────────────────────────────┴───────┴────────┘
